In [ ]:
import os
import sys
from abc import ABC, abstractmethod
from typing import Generator

import jedi
from jedi.api.classes import Completion, Name, Signature
from jedi.api.errors import SyntaxError


class ResponseTypes:
    List = list
    Generator = Generator
    Name = Name
    Signature = Signature
    Completion = Completion
    SyntaxError = SyntaxError


class AbstractResponse(ABC):
    @abstractmethod
    def print_response(self, completions: ResponseTypes.Name | ResponseTypes.Generator | ResponseTypes.List | ResponseTypes.Signature):
        pass


class NameType(AbstractResponse):
    def print_response(self, completions: ResponseTypes.Name):
        print(completions)
        print(f"completions.full_name: {completions.full_name}")
        print(f"completions.description: {completions.description}")


class GeneratorType(AbstractResponse):
    def print_response(self, completions: ResponseTypes.Generator):
        print(f"CompleteSearch Generator: {completion}")


class ListType(AbstractResponse):
    """ Completions / Responses Generator """
    def print_response(self, completions: ResponseTypes.List):
        if len(completions) == 0:
            return
        for completion in completions:
            yield completion


class SignatureType(AbstractResponse):
    def print_response(self, index, completion: ResponseTypes.Signature):
        print(f"Signatures[{index}]: {completion}")


curr_py_env = sys.path
project_dir = os.path.join(os.path.dirname(os.path.dirname(__file__)))
file_path = os.path.join(project_dir, "src", "llmcoder", "analyze", "GPTReviewAnalyzer.py")
curr_source_code = ""
with open(file_path, "r") as file:
    curr_source_code = file.read()


# source: str = """import json
# json.load("""
# source_lines: list[str] = source.splitlines()
source_line: int = 82
# source_col: int = 42 # .chat : instance
source_col: int = 62 # create() : def

# from pipreqs import pipreqs

# pipreqs.project_dir = project_dir
# import subprocess

# subprocess.run(["pipreqs", "--print", project_dir])
# jedi.preload_module(["os", "re", "openai", "llmcoder"])
# project = jedi.Project(path=project_dir, environment_path="/home/kushal/.config/python.env/llmcoder.env")
# script = jedi.Script(curr_source_code, path=file_path, project=project, environment=project.get_environment())
signatures = []
script = jedi.Script(curr_source_code, path=file_path)
# completions = script.complete_search()
# completions = script.infer(source_line, source_col)
# completions = script.get_signatures(source_line, source_col)
# completions = script.goto(source_line, source_col)
# completions = script.infer()
# completions = script.get_references()
# completions = script.complete(source_line, source_col)
# completions = script.get_context(source_line, source_col)
# completions = script.get_syntax_errors()
# completions = script.extract_function()
completions = script.get_names()

print("="*50)
print(f"type(completions): {type(completions)}")
print("-"*20)
if isinstance(completions, ResponseTypes.Name):
    # Get context
    print(f"Get context: {completions}")
    print("-"*20)
elif isinstance(completions, ResponseTypes.List):
    if len(completions) == 0:
        print("Empty Responses")
        print("-"*20)
    for index, completion in enumerate(completions):
        if isinstance(completion, ResponseTypes.Name):
            if hasattr(completion, completion.full_name):
                print(f"Yes it has!!")
            # Get references
            print(f"Completion[{index}].line: {completion.line}")
            print(f"Completion[{index}].column: {completion.column}")
            print(f"Completion[{index}].module_path: {completion.module_path}")
            print(f"References[{index}]: {completion}")
            # if "def" in completion.description:
            #     temp_script_path = os.path.join(*(completion.full_name.split(".")[:-1]))+".py"
            #     splitted = os.path.split(completion.module_path)
            #     splitted = os.path.split(splitted[0])
            #     print(splitted)
            #     splitted = os.path.split(os.path.join(*splitted))
            #     print(splitted[0:splitted=="llmcoder"])
            #     temp_module_path = os.path.join(*splitted[:splitted=="llmcoder"])
            #     print(temp_module_path)
            #     temp_source_code = open(temp_script_path, "r")
            #     temp_script = jedi.Script(code=temp_source_code, path=temp_script_path)
            #     temp_names = temp_script.get_names()
            #     for temp_name in temp_names:
            #         print(f"temp_name[{index}].get_line_code: {temp_name.get_line_code(after=temp_name.get_definition_end_position()[0]-temp_name.get_definition_start_position()[0])}")
            # if "def" in completion.description or "class" in completion.description:
            if "def" in completion.description or "class" in completion.description:
                print(f"Completion[{index}].get_line_code: {completion.get_line_code(after=completion.get_definition_end_position()[0]-completion.get_definition_start_position()[0])}")
            # print(f"Completion[{index}].get_definition_end_position: {completion.get_definition_end_position()}")
            # print(f"Completion[{index}].get_definition_start_position: {completion.get_definition_start_position()}")
            # print(f"Completion[{index}].get_type_hint: {completion.get_type_hint()}")
            # print(f"Completion[{index}].is_def: {('def' in completion.description)}")
            # print(f"Completion[{index}].is_definition: {completion.is_definition()}")
            if "def" in completion.description or "class" in completion.description:
                goto_definitions = completion.goto(follow_imports=True, follow_builtin_imports=True)
                print(f"Completions[{index}].goto.line: {goto_definitions[0].line}")
                print(f"Completions[{index}].goto.get_line_code: {goto_definitions[0].get_line_code(after=goto_definitions[0].get_definition_end_position()[0]-goto_definitions[0].get_definition_start_position()[0])}")
                print(f"Completions[{index}].goto: {completion.get_signatures()}")
            # print(f"Completions[{index}].full_name: {completion.full_name}")
            # print(f"Completions[{index}].description: {completion.description}")
            print("-"*20)
        elif isinstance(completion, ResponseTypes.Signature):
            # Signatures
            print(f"Signatures[{index}]: {completion.docstring()}")
            print("-"*20)
        elif isinstance(completion, ResponseTypes.Generator):
            # Complete search
            print(f"Complete Search[{index}]: {next(completion)}")
            print("-"*20)
        elif isinstance(completion, ResponseTypes.SyntaxError):
            # Syntax errors
            print(f"SyntaxErrors[{index}]: {completion}")
            print(f"Error line: {completion.line}")
            print(f"Error column: {completion.column}")
            print(f"Error until line: {completion.until_line}")
            print(f"Error until column: {completion.until_column}")
            print(f"Error message: {completion.get_message()}")
            print("-"*20)
        elif isinstance(completion, ResponseTypes.Completion):
            # Get completion suggestions
            print(f"Completions[{index}]: {completion}")
            print(f"Completions[{index}].complete: {completion.complete}")
            print(f"Completions[{index}].name: {completion.name}")
            print("-"*20)
        else:
            print(f"Completions[{index}]: {completion}")
            print(f"Completions[{index}].complete: {completion.complete}")
            print(f"Completions[{index}].name: {completion.name}")
            print("-"*20)

print(f"Script: {script}")
print("="*50)